In [1]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [2]:
from comm_agents.data.reference_experiments import RefExperimentMass, RefExperimentCharge
from comm_agents.data.optimal_answers import get_alpha_star, get_phi_star
from ipywidgets import interact
import numpy as  np

In [3]:
# experiment parameters
GOLF_HOLE_LOC_M = .1
GOLF_HOLE_LOC_C = .1
TOLERANCE = .01
PARAM_DICT = dict(
    m=[1e-20, 1e-20],
    q=[1e-16, -1e-15],
    m_ref_m=2e-20,
    q_ref=[-1e-17, -1e-17],
    v_ref_m=2,
    m_ref_c=2e-20,
    v_ref_c=0,
    N=10,
    alpha=[0, 0],
    phi=[0, 0],
    dt=.01,
    d=.1,
    is_golf_game=False,
    gravity=True)
M_RANGES = [1e-20, 5e-20]
Q0_RANGE = [1e-16, 2e-16]
Q1_RANGE = [-1e-15, -2e-15]
V_REF_RANGE = [1, 2]

In [4]:
alpha_range_re1 = np.linspace(0, 0.6, 50)
m_range = list(np.linspace(1e-20, 1e-19, 50))
alpha_range_re2 = np.linspace(0.5, 1, 20)
q_range_re2_1 = np.linspace(1e-16, 2e-16, 50)
q_range_re2_2 = np.linspace(-1e-15, -2e-15, 50)
v_ref_range = np.linspace(1, 2, 50)
N_reange = [10, 100, 1000]
dt_range = [.01, .001, .0001]

In [10]:
def get_re1_plot(alpha1, alpha2, m1, m2, v_ref, N, dt, alpha_star):
    rem = RefExperimentMass(**PARAM_DICT)
    rem.N = N
    rem.dt = dt
    rem.angle = np.array([alpha1*np.pi, alpha2*np.pi])
    rem.m = np.array([m1, m2])
    rem.v_ref = v_ref
    if alpha_star:
        rem.angle, loss = get_alpha_star(rem)
        hio = rem.check_for_hole_in_one(golf_hole_loc=GOLF_HOLE_LOC_M, tolerance=TOLERANCE)
        print(f'Loss: {loss}, Hole in one: {hio}')
    rem.set_initial_state()
    rem.run()
    from comm_agents.data.data_handler import RefExpDataset
    ds = RefExpDataset()
    rem.visualize(golf_hole_loc=GOLF_HOLE_LOC_M, tolerance=TOLERANCE)
    
interact(get_re1_plot, alpha1=alpha_range_re1, alpha2=alpha_range_re1,
         m1=m_range, m2=m_range, N=N_reange, alpha_star=False, v_ref=v_ref_range,
        dt=dt_range)

interactive(children=(Dropdown(description='alpha1', options=(0.0, 0.012244897959183673, 0.024489795918367346,…

<function __main__.get_re1_plot(alpha1, alpha2, m1, m2, v_ref, N, dt, alpha_star)>

In [8]:
def get_re2_plot(m1, m2, v_ref, phi1=.5, phi2=.63, q1=4e-12, q2=-3e-12, N=100, 
                 dt=0.01, phi_star=False, y_cap=True):
    req = RefExperimentCharge(**PARAM_DICT)
    req.is_golf_game = True
    req.N = N
    req.dt = dt
    req.y_cap = y_cap
    req.angle = np.array([phi1*np.pi, phi2*np.pi])
    req.q = np.array([q1, q2])
    req.m = np.array([m1, m2])
    req.v_ref = v_ref
    if phi_star:
        req.angle, loss = get_phi_star(req)
        hio = req.check_for_hole_in_one(golf_hole_loc=GOLF_HOLE_LOC_C, tolerance=TOLERANCE)
        print(f'Loss: {loss}, Hole in one: {hio}')
    req.set_initial_state()
    req.run()
#     print(f'Hidden states \n{ds.hidden_states[0:3]}')
#     print(f'Questions \n{ds.questions[0:3]}')
#     print(f'Answers \n{ds.opt_answers[0:3]/np.pi}')
    req.visualize(golf_hole_loc=GOLF_HOLE_LOC_C)
    print(req.q_ref)

interact(get_re2_plot, phi1=alpha_range_re2, phi2=alpha_range_re2,
        q1=q_range_re2_1, q2=q_range_re2_2,
         m1=m_range, m2=m_range,
         N=N_reange, phi_star=False,
         v_ref=v_ref_range,
         y_cap=True,
        dt=dt_range)

interactive(children=(Dropdown(description='m1', options=(1e-20, 1.183673469387755e-20, 1.36734693877551e-20, …

<function __main__.get_re2_plot(m1, m2, v_ref, phi1=0.5, phi2=0.63, q1=4e-12, q2=-3e-12, N=100, dt=0.01, phi_star=False, y_cap=True)>

In [11]:
import pandas as pd
DATA_PATH = './data/training/test.csv'

df_check = pd.read_csv(DATA_PATH)
df_check.loc[:, ['alpha_star0', 'alpha_star1', 'phi_star0', 'phi_star1']] = \
    df_check[['alpha_star0', 'alpha_star1', 'phi_star0', 'phi_star1']] / np.pi
df_check.head()

,m0,m1,q0,q1,o_a_0_0,o_a_0_1,o_a_0_2,o_a_0_3,o_a_0_4,o_a_0_5,...,o_b_1_8,o_b_1_9,m_ref_a,v_ref_a,m_ref_b,v_ref_b,alpha_star0,alpha_star1,phi_star0,phi_star1
0,4.647805e-20,3.363276e-20,4.715559e-16,-3.664115e-16,0.0,0.010463,0.020925,0.031388,0.041851,0.052314,...,-0.000313,-0.000396,2.000000e-20,1.738854,2.000000e-20,1.917186,0.178110,0.099106,0.533421,0.529859
1,2.899667e-20,1.166211e-20,4.741920e-16,-4.048057e-16,0.0,0.016044,0.032088,0.048132,0.064175,0.080219,...,-0.000995,-0.001259,2.000000e-20,1.965237,2.000000e-20,1.019186,0.063111,0.024930,0.626321,0.632407
2,1.390255e-20,1.189017e-20,6.148877e-16,-6.139955e-16,0.0,0.020255,0.040511,0.060766,0.081022,0.101277,...,-0.001478,-0.001869,2.000000e-20,1.716780,2.000000e-20,1.856558,0.038839,0.034354,0.569423,0.571589
3,4.699369e-20,2.583949e-20,3.235056e-16,-4.704748e-16,0.0,0.009877,0.019753,0.029630,0.039506,0.049383,...,-0.000523,-0.000661,2.000000e-20,1.654173,2.000000e-20,1.281428,0.249900,0.077157,0.567354,0.557170
4,3.487187e-20,3.458264e-20,5.677379e-16,-6.160624e-16,0.0,0.010981,0.021961,0.032942,0.043922,0.054903,...,-0.000512,-0.000647,2.000000e-20,1.506311,2.000000e-20,1.193513,0.150200,0.150200,0.726993,0.723112


In [13]:
i = 0

In [63]:
i +=1
m0 = df_check.m0[i]
m1 = df_check.m1[i]
q0 = df_check.q0[i]
q1 = df_check.q1[i]
v_ref_c = df_check.v_ref_b[i]
phi0 = df_check.phi_star0[i]
phi1 = df_check.phi_star1[i]
req = RefExperimentCharge( m=[m0, m1], q=[q0, q1], m_ref_c=2e-20,
                 v_ref_c=v_ref_c, q_ref=[None, None], d=.1, N=1000,
                          phi=[phi0*np.pi, phi1*np.pi],
                 dt=.001, is_golf_game=True, y_cap=True)
req.run()
req.visualize()

In [7]:
# some quick feasibility checks
import pandas as pd
DATA_PATH = './data/training/test.csv'

df_check = pd.read_csv(DATA_PATH)
df_check.loc[:, ['alpha_star0', 'alpha_star1', 'phi_star0', 'phi_star1']] = \
    df_check[['alpha_star0', 'alpha_star1', 'phi_star0', 'phi_star1']] / np.pi
df_check.head()

,m0,m1,q0,q1,o_a_0_0,o_a_0_1,o_a_0_2,o_a_0_3,o_a_0_4,o_a_0_5,...,o_b_1_8,o_b_1_9,m_ref_a,v_ref_a,m_ref_b,v_ref_b,alpha_star0,alpha_star1,phi_star0,phi_star1
0,4.647805e-20,3.363276e-20,4.715559e-16,-3.664115e-16,0.0,0.010463,0.020925,0.031388,0.041851,0.052314,...,-0.000313,-0.000396,2.000000e-20,1.738854,2.000000e-20,1.917186,0.178110,0.099106,0.533421,0.529859
1,2.899667e-20,1.166211e-20,4.741920e-16,-4.048057e-16,0.0,0.016044,0.032088,0.048132,0.064175,0.080219,...,-0.000995,-0.001259,2.000000e-20,1.965237,2.000000e-20,1.019186,0.063111,0.024930,0.626321,0.632407
2,1.390255e-20,1.189017e-20,6.148877e-16,-6.139955e-16,0.0,0.020255,0.040511,0.060766,0.081022,0.101277,...,-0.001478,-0.001869,2.000000e-20,1.716780,2.000000e-20,1.856558,0.038839,0.034354,0.569423,0.571589
3,4.699369e-20,2.583949e-20,3.235056e-16,-4.704748e-16,0.0,0.009877,0.019753,0.029630,0.039506,0.049383,...,-0.000523,-0.000661,2.000000e-20,1.654173,2.000000e-20,1.281428,0.249900,0.077157,0.567354,0.557170
4,3.487187e-20,3.458264e-20,5.677379e-16,-6.160624e-16,0.0,0.010981,0.021961,0.032942,0.043922,0.054903,...,-0.000512,-0.000647,2.000000e-20,1.506311,2.000000e-20,1.193513,0.150200,0.150200,0.726993,0.723112


In [22]:
i=0

In [28]:
alpha0

0.28301128733017245

In [9]:
df_check[[c for c in df_check.columns if 'o0_b' in c]]

""
0
1
2
3
4
...
513
514
515
516


In [33]:
df_check

,m0,m1,q0,q1,o0_a_0,o0_a_1,o0_a_2,o0_a_3,o0_a_4,o0_a_5,...,o1_b_8,o1_b_9,m_ref_a,v_ref_a,m_ref_b,v_ref_b,alpha_star0,alpha_star1,phi_star0,phi_star1
0,4.633908e-20,1.906103e-20,1.219605e-16,-1.008854e-15,0.0,0.009969,0.019937,0.029906,0.039875,0.049844,...,-0.001515,-0.001915,2.000000e-20,1.653289,2.000000e-20,1.883145,0.250000,0.055634,0.524394,0.520660
1,3.151944e-20,1.393146e-20,1.980285e-16,-1.166511e-15,0.0,0.013317,0.026634,0.039952,0.053269,0.066586,...,-0.002390,-0.003019,2.000000e-20,1.715241,2.000000e-20,1.768716,0.092312,0.038838,0.546843,0.546333
2,3.123298e-20,2.724422e-20,1.179788e-16,-1.075147e-15,0.0,0.014167,0.028335,0.042502,0.056670,0.070837,...,-0.001131,-0.001430,2.000000e-20,1.814592,2.000000e-20,1.829524,0.081381,0.068986,0.523798,0.523036
3,2.843531e-20,4.045260e-20,1.539548e-16,-1.258396e-15,0.0,0.015599,0.031197,0.046796,0.062394,0.077993,...,-0.000892,-0.001128,2.000000e-20,1.888803,2.000000e-20,1.898299,0.065045,0.108409,0.533033,0.536382
4,2.762113e-20,2.318476e-20,1.910677e-16,-1.677997e-15,0.0,0.013473,0.026946,0.040420,0.053893,0.067366,...,-0.002068,-0.002613,2.000000e-20,1.604023,2.000000e-20,1.186415,0.090085,0.073025,0.668092,0.662720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,1.633190e-20,1.539358e-20,1.198750e-16,-1.102032e-15,0.0,0.020491,0.040982,0.061474,0.081965,0.102456,...,-0.002046,-0.002585,2.000000e-20,1.861209,2.000000e-20,1.283289,0.037315,0.035821,0.548857,0.549123
505,1.375134e-20,1.041888e-20,1.577665e-16,-1.425324e-15,0.0,0.018597,0.037194,0.055791,0.074387,0.092984,...,-0.003886,-0.004902,2.000000e-20,1.569172,2.000000e-20,1.559541,0.045221,0.037315,0.557944,0.562835
506,3.487172e-20,2.937758e-20,1.690140e-16,-1.088408e-15,0.0,0.011097,0.022193,0.033290,0.044387,0.055483,...,-0.001062,-0.001343,2.000000e-20,1.522231,2.000000e-20,1.777930,0.147675,0.110774,0.537329,0.535947
507,4.807636e-20,3.701136e-20,1.191293e-16,-1.763968e-15,0.0,0.011696,0.023392,0.035087,0.046783,0.058479,...,-0.001365,-0.001725,2.000000e-20,1.990523,2.000000e-20,1.544775,0.127713,0.083526,0.564964,0.558532


In [66]:
1/df_check.o1_a_9.max() * df_check.o1_a_9.max()

1.0

In [61]:
i +=1
m0 = df_check.m0[i]
m1 = df_check.m1[i]
q0 = df_check.q0[i]
q1 = df_check.q1[i]
v_ref_m = df_check.v_ref_a[i]
alpha0 = df_check.alpha_star0[i]
alpha1 = df_check.alpha_star1[i]
rem = RefExperimentMass(m=[m0, m1], m_ref_m=2e-20, v_ref_m=v_ref_m, N=1000,
                 alpha=[alpha0, alpha1], dt=.001, gravity=True)
rem.angle = np.array([alpha0*np.pi, alpha1*np.pi])
rem.run()
rem.visualize(GOLF_HOLE_LOC_M, TOLERANCE)

In [62]:
 df_check.m0[i],  df_check.m1[i]

(1.3436429859232451e-20, 2.4051055722030438e-20)

In [30]:
df_sampels = pd.read_csv('data/reference_experiment_dat.csv')

In [31]:
df_sampels.alpha_star0.max() / np.pi

0.14264174223876083

In [32]:
df_sampels[0:1].to_dict()

{'m0': {0: 1.3787796896581534e-20},
 'm1': {0: 3.098626843255872e-20},
 'q0': {0: 1.099232064298954e-16},
 'q1': {0: -1.4092251778554867e-15},
 'o0_0_0': {0: 0.0},
 'o0_0_1': {0: 0.02092927320893955},
 'o0_0_2': {0: 0.0418585464178791},
 'o0_0_3': {0: 0.06278781962681865},
 'o0_0_4': {0: 0.0837170928357582},
 'o0_0_5': {0: 0.10464636604469776},
 'o0_0_6': {0: 0.12557563925363732},
 'o0_0_7': {0: 0.14650491246257688},
 'o0_0_8': {0: 0.16743418567151644},
 'o0_0_9': {0: 0.18836345888045603},
 'o0_1_0': {0: 0.0},
 'o0_1_1': {0: 0.01386949965385468},
 'o0_1_2': {0: 0.02773899930770936},
 'o0_1_3': {0: 0.04160849896156405},
 'o0_1_4': {0: 0.05547799861541872},
 'o0_1_5': {0: 0.0693474982692734},
 'o0_1_6': {0: 0.08321699792312809},
 'o0_1_7': {0: 0.09708649757698276},
 'o0_1_8': {0: 0.11095599723083746},
 'o0_1_9': {0: 0.12482549688469215},
 'o1_0_0': {0: 0.0},
 'o1_0_1': {0: 3.5836386088983146e-06},
 'o1_0_2': {0: 1.4335068161835401e-05},
 'o1_0_3': {0: 3.225583024265275e-05},
 'o1_0_4': {

In [10]:
import pandas as pd
DATA_PATH = './data/reference_experiment_dat_1000_new_new_new.csv'

df_check = pd.read_csv(DATA_PATH)
df_check

,m0,m1,q0,q1,o0_a_0,o0_a_1,o0_a_2,o0_a_3,o0_a_4,o0_a_5,...,o1_b_8,o1_b_9,m_ref_a,v_ref_a,m_ref_b,v_ref_b,alpha_star0,alpha_star1,phi_star0,phi_star1
0,4.633908e-20,1.906103e-20,1.219605e-16,-1.008854e-15,0.0,0.009969,0.019937,0.029906,0.039875,0.049844,...,-0.001515,-0.001915,2.000000e-20,1.653289,2.000000e-20,1.883145,0.785398,0.174778,1.647433,1.635702
1,3.151944e-20,1.393146e-20,1.980285e-16,-1.166511e-15,0.0,0.013317,0.026634,0.039952,0.053269,0.066586,...,-0.002390,-0.003019,2.000000e-20,1.715241,2.000000e-20,1.768716,0.290007,0.122014,1.717959,1.716356
2,3.123298e-20,2.724422e-20,1.179788e-16,-1.075147e-15,0.0,0.014167,0.028335,0.042502,0.056670,0.070837,...,-0.001131,-0.001430,2.000000e-20,1.814592,2.000000e-20,1.829524,0.255667,0.216727,1.645559,1.643166
3,2.843531e-20,4.045260e-20,1.539548e-16,-1.258396e-15,0.0,0.015599,0.031197,0.046796,0.062394,0.077993,...,-0.000892,-0.001128,2.000000e-20,1.888803,2.000000e-20,1.898299,0.204344,0.340577,1.674573,1.685094
4,2.762113e-20,2.318476e-20,1.910677e-16,-1.677997e-15,0.0,0.013473,0.026946,0.040420,0.053893,0.067366,...,-0.002068,-0.002613,2.000000e-20,1.604023,2.000000e-20,1.186415,0.283011,0.229416,2.098874,2.081995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,1.633190e-20,1.539358e-20,1.198750e-16,-1.102032e-15,0.0,0.020491,0.040982,0.061474,0.081965,0.102456,...,-0.002046,-0.002585,2.000000e-20,1.861209,2.000000e-20,1.283289,0.117229,0.112534,1.724286,1.725122
505,1.375134e-20,1.041888e-20,1.577665e-16,-1.425324e-15,0.0,0.018597,0.037194,0.055791,0.074387,0.092984,...,-0.003886,-0.004902,2.000000e-20,1.569172,2.000000e-20,1.559541,0.142065,0.117227,1.752833,1.768199
506,3.487172e-20,2.937758e-20,1.690140e-16,-1.088408e-15,0.0,0.011097,0.022193,0.033290,0.044387,0.055483,...,-0.001062,-0.001343,2.000000e-20,1.522231,2.000000e-20,1.777930,0.463935,0.348008,1.688068,1.683729
507,4.807636e-20,3.701136e-20,1.191293e-16,-1.763968e-15,0.0,0.011696,0.023392,0.035087,0.046783,0.058479,...,-0.001365,-0.001725,2.000000e-20,1.990523,2.000000e-20,1.544775,0.401221,0.262405,1.774888,1.754679


In [11]:
df_check.alpha_star0.max()

0.7853981633974483

In [ ]:
rem = RefExperimentMass(**PARAM_DICT)
rem.m = []